In [ ]:
import keras
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar10
from keras import regularizers
from keras.callbacks import LearningRateScheduler
import numpy as np

(x_train, y_train), (x_test, y_test) = cifar10.load_data()
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')

# FOR PHASE 1 of Training
def lr_schedule_1(epoch): 
    lrate = 0.001
    if epoch > 75:
        lrate = 0.0005
    elif epoch > 100:
        lrate = 0.0003        
    return lrate

# FOR PHASE 2 of Training
def lr_schedule_2(epoch):  
    lrate = 0.001
    if epoch > 30:
        lrate = 0.0005
    elif epoch > 45:
        lrate = 0.0003        
    return lrate

# FOR PHASE 3 of Training
def lr_schedule_3(epoch): 
    lrate = 0.001
    #if epoch > 15:
    #    lrate = 0.0005
    #elif epoch > 25:
    #    lrate = 0.0003        
    return lrate
    
#Data normalization as per CIFAR-10 dataset instructions - https://www.cs.toronto.edu/~kriz/cifar.html
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 10
y_train = np_utils.to_categorical(y_train,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)


In [ ]:
#Construct CNN

weight_decay = 1e-4
model = Sequential()

# Add convolutional layers
model.add(Conv2D(32, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.3))

model.add(Flatten())
# Add softmax (output) layer
model.add(Dense(num_classes, activation='softmax'))

#data augmentation
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    )
datagen.fit(x_train)

#training
batch_size = 64

opt_rms = keras.optimizers.RMSprop(lr=0.001,decay=1e-6)
model.compile(loss='categorical_crossentropy', optimizer=opt_rms, metrics=['accuracy'])

# Only one model.fit call of the 3 listed below must be uncommented while execution

# For Phase 1: 125 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=125,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule_1)])

# For Phase 2: 60 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=60,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule_2)])

# For Phase 3: 30 epochs
model.fit((datagen.flow(x_train, y_train, batch_size=batch_size)),steps_per_epoch=x_train.shape[0] // batch_size,epochs=30,verbose=1,validation_data=(x_test,y_test),callbacks=[LearningRateScheduler(lr_schedule)])

In [ ]:
y_pred_test = model.predict(x_test)
#y_pred_test
joint_prob = np.zeros((10,10)) #for CIFAR 10 (i,j) - state (true label) i, action(prediction prob) j
for i in range(10):
  idx  = np.where(np.argmax(y_test,1)==i)
  for j in range(10):
    joint_prob[i,j] = np.mean(y_pred_test[idx,j])

joint_prob = joint_prob*0.1 # prior pmf over the set of true image classes in CIFAR-10  is uniform.

# joint_prob[x,a] is the joint probability that the true image class is 'x' and predicted image class is 'a'
joint_prob

array([[0.04169796, 0.00933388, 0.0096306 , 0.00444369, 0.00383221,
        0.00356222, 0.0017174 , 0.0040374 , 0.01148615, 0.01025848],
       [0.00249347, 0.06624796, 0.00097874, 0.00164424, 0.00073972,
        0.00138457, 0.00123213, 0.00154341, 0.0037154 , 0.02002035],
       [0.00658071, 0.00337976, 0.02560672, 0.0113429 , 0.01246649,
        0.01483983, 0.00884424, 0.00932769, 0.0023523 , 0.00525935],
       [0.00196926, 0.00399997, 0.0071328 , 0.02995055, 0.00691522,
        0.02377362, 0.00998317, 0.00880662, 0.00143384, 0.00603495],
       [0.003643  , 0.00265944, 0.01348804, 0.01165847, 0.0256724 ,
        0.01220714, 0.01197101, 0.01341827, 0.00159274, 0.00368949],
       [0.0012814 , 0.00208289, 0.00697288, 0.01958069, 0.00544615,
        0.04316215, 0.00576842, 0.01130543, 0.00094132, 0.00345868],
       [0.00133819, 0.00427115, 0.00683609, 0.01370223, 0.00878531,
        0.01010698, 0.04218492, 0.00582403, 0.00109087, 0.00586025],
       [0.00178331, 0.00231202, 0.0041550

In [ ]:
#Code structure credits :  https://github.com/abhijeet3922/Object-recognition-CIFAR-10/blob/master/cifar10_90.py